# 1. 토큰화

### 품사 태깅 이유
+ game, play 등의 의미없는 단어가 너무 많음. => 일일히 걸러주자
+ 품사 태깅 이유 : 키워드 추출 할때 부사와 같은 꾸며주는 단어가 많이 나와서 추출하는데 의미가 없다고 생각이 들었음
+ 동사, 명사와 같은 의미있는 단어만 뽑아주기 위해 품사 태깅 사용

+ 원형, 표제어 추출하면 품사 태깅 안해도 괜찮을 듯

In [3]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk import pos_tag
import pandas as pd
import re

# 불용어와 어간 추출, 표제어 추출 초기화
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()  # 어간 추출기
lemmatizer = WordNetLemmatizer()  # 표제어 추출기

# 정의한 불용어 리스트
stop_text = ['game', 'warframe', 'destiny', 'destiny2', 'thefirstdescendant', 'gameplay', 'descendant', 'tfd', 'first', 'play']

# 전처리 함수 정의
def preprocess_review(text):
    # 1. 소문자 변환
    text = text.lower()
    # 2. 특수문자 및 숫자 제거
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # 3. 토큰화
    tokens = word_tokenize(text)
    # 4. 품사 태깅
    tagged_tokens = pos_tag(tokens)
    # 5. 명사(NN), 동사(VB), 형용사(JJ)만 필터링하고 표제어 추출
    filtered_tokens = [
        lemmatizer.lemmatize(word)  # 표제어 추출
        for word, tag in tagged_tokens
        if tag.startswith(('NN', 'VB', 'JJ')) and (word not in stop_words) and (word not in stop_text)
    ]
    return filtered_tokens

df = pd.read_csv('Warframe_reviews.csv')

# 전처리 적용
df['cleaned_text'] = df['review_text'].apply(preprocess_review)
print(df[['review_text', 'cleaned_text']].head())


                                         review_text  \
0  Dealing 5 billion damage to a 100 health point...   
1  Welcome to Warframe. Play the same mission 231...   
2  The best part of this game is figuring out wha...   
3  I hope you guys from The First Descendant, Des...   
4  Once Cross save is available, I will start pla...   

                                        cleaned_text  
0  [dealing, damage, health, point, enemy, crucia...  
1  [welcome, mission, time, peepeepoopoo, blueprint]  
2  [best, part, figuring, want, next, there, wors...  
3  [hope, guy, division, read, faq, raid, raid, u...  
4  [cross, save, available, start, playing, ill, ...  


In [ ]:
# 품사 태깅 테스트 (동사 많이 나온 단어 추출)
# from collections import Counter

# tagged_tokens = nltk.pos_tag(all_tokens)
# nouns = [word for word, pos in tagged_tokens if pos.startswith('VB')]

# counter = Counter(nouns)
# top_30_nn = counter.most_common(30)
# print(top_30_nn)

# 2. 임베딩

In [4]:
# 토큰 리스트를 문자열로 변환
df['cleaned_text'] = df['cleaned_text'].apply(lambda tokens: ' '.join(tokens)) 

# 결과 확인
print(df[['review_text', 'cleaned_text']].head())

                                         review_text  \
0  Dealing 5 billion damage to a 100 health point...   
1  Welcome to Warframe. Play the same mission 231...   
2  The best part of this game is figuring out wha...   
3  I hope you guys from The First Descendant, Des...   
4  Once Cross save is available, I will start pla...   

                                        cleaned_text  
0  dealing damage health point enemy crucial plot...  
1        welcome mission time peepeepoopoo blueprint  
2  best part figuring want next there worst part ...  
3  hope guy division read faq raid raid used hect...  
4  cross save available start playing ill say gre...  


In [5]:
# Load model directly
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("mixedbread-ai/mxbai-embed-large-v1")
model = AutoModel.from_pretrained("mixedbread-ai/mxbai-embed-large-v1")

def get_embeddings_in_batches(texts, tokenizer, model, batch_size=16):
    all_embeddings = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        tokens = tokenizer(
            batch_texts,
            padding=True,
            truncation=True,
            return_tensors="pt",
            max_length=512
        )
        with torch.no_grad():
            outputs = model(**tokens)
            embeddings = outputs.last_hidden_state.mean(dim=1)  # 평균 풀링
            all_embeddings.append(embeddings.cpu().numpy())  # CPU 메모리로 이동
    return np.vstack(all_embeddings)  # 배열 합치기

# 임베딩 실행
texts = df['cleaned_text'].tolist() 
embeddings = get_embeddings_in_batches(texts, tokenizer, model, batch_size=16)

print("임베딩 크기:", embeddings.shape)

임베딩 크기: (2780, 1024)


# 3. 감정 분석

In [8]:
print(tokenizer.model_max_length)


512


In [10]:
from transformers import pipeline

# 감정 분석 파이프라인 로드
pipe = pipeline("text-classification", model="cardiffnlp/twitter-roberta-base-sentiment-latest")

# 감정 분석 수행
def analyze_sentiment(texts):
    sentiments = []
    for text in texts:
        # 감정 분석 수행 및 결과 추출
        sentiment_result = pipe(text[:512])  # 길이 초과 방지를 위해 512 토큰 제한
        sentiments.append(sentiment_result[0]['label'])
    return sentiments

# 감정 분석 실행
df['sentiment'] = analyze_sentiment(df['cleaned_text'].tolist())

# 결과 확인
print(df[['cleaned_text', 'sentiment']].head())

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use mps:0


                                        cleaned_text sentiment
0  dealing damage health point enemy crucial plot...   neutral
1        welcome mission time peepeepoopoo blueprint  positive
2  best part figuring want next there worst part ...   neutral
3  hope guy division read faq raid raid used hect...   neutral
4  cross save available start playing ill say gre...  positive


In [14]:
# 긍정/부정 리뷰 분리
positive_reviews = df[df['sentiment'] == 'positive']
negative_reviews = df[df['sentiment'] == 'negative']
neutral_reviews = df[df['sentiment'] == 'neutral']

print(f"긍정 리뷰 개수: {len(positive_reviews)}")
print(f"부정 리뷰 개수: {len(negative_reviews)}")
print(f"중립 리뷰 개수: {len(neutral_reviews)}")

긍정 리뷰 개수: 1406
부정 리뷰 개수: 334
중립 리뷰 개수: 1040


# 4. 토픽 모델링

In [15]:
from bertopic import BERTopic

# 긍정 리뷰 토픽 모델링
positive_texts = positive_reviews['cleaned_text'].tolist()
topic_model_positive = BERTopic()
topics_positive, probs_positive = topic_model_positive.fit_transform(positive_texts)

# 부정 리뷰 토픽 모델링
negative_texts = negative_reviews['cleaned_text'].tolist()
topic_model_negative = BERTopic()
topics_negative, probs_negative = topic_model_negative.fit_transform(negative_texts)

# 긍정 토픽 확인
print("긍정 리뷰 주요 토픽:", topic_model_positive.get_topic_info())

# 부정 토픽 확인
print("부정 리뷰 주요 토픽:", topic_model_negative.get_topic_info())


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


긍정 리뷰 주요 토픽:     Topic  Count                                    Name  \
0      -1    485                   -1_free_get_fun_great   
1       0     60          0_received_product_free_played   
2       1     54             1_grind_grinding_grindy_fun   
3       2     52          2_weapon_mission_warframes_get   
4       3     52                   3_uwu_antipay_lol_win   
5       4     50                    4_good_owo_gamee_sir   
6       5     45                 5_pc_devs_content_steam   
7       6     41              6_space_ninja_robot_weapon   
8       7     39             7_awesome_cool_awsome_great   
9       8     35        8_coop_character_get_multiplayer   
10      9     32                9_hour_seems_played_good   
11     10     31                 10_player_get_time_feel   
12     11     30         11_graphic_story_nice_beautiful   
13     12     30         12_free_anthem_better_excellent   
14     13     28     13_shooter_person_third_thirdperson   
15     14     28           

In [16]:
# 긍정 리뷰 토픽 내 주요 단어
for topic in range(len(topic_model_positive.get_topic_info())):
    print(f"Topic {topic}: {topic_model_positive.get_topic(topic)}")

# 부정 리뷰 토픽 내 주요 단어
for topic in range(len(topic_model_negative.get_topic_info())):
    print(f"Topic {topic}: {topic_model_negative.get_topic(topic)}")


Topic 0: [('received', 0.16323244093185652), ('product', 0.16246694255802344), ('free', 0.08430609281335033), ('played', 0.024457257076038328), ('best', 0.021873688933877956), ('ive', 0.021259029959625024), ('fun', 0.020459050747535144), ('give', 0.019651447829026766), ('good', 0.01891327363944329), ('time', 0.017379659029284813)]
Topic 1: [('grind', 0.5100401981689908), ('grinding', 0.08017773360100601), ('grindy', 0.03745232663626611), ('fun', 0.036388534539078116), ('good', 0.026557271306499125), ('shiny', 0.02471847300659537), ('fest', 0.023728127693038968), ('love', 0.02260751678082329), ('grindfest', 0.020660010348720514), ('great', 0.018363788091615305)]
Topic 2: [('weapon', 0.02318770882764612), ('mission', 0.01721793683229958), ('warframes', 0.01621569077124849), ('get', 0.015906892305297984), ('great', 0.015090145770033092), ('story', 0.01456450572206172), ('space', 0.014091394715671817), ('graphic', 0.013907943657818434), ('rpg', 0.013899827097796488), ('system', 0.013478889

In [19]:
# 긍정 리뷰 토픽 관계도 시각화
topic_model_positive.visualize_topics()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'customdata': array([[0, 'received | product | free | played | best', 60],
                                   [1, 'grind | grinding | grindy | fun | good', 54],
                                   [2, 'weapon | mission | warframes | get | great', 52],
                                   [3, 'uwu | antipay | lol | win | ', 52],
                                   [4, 'good | owo | gamee | sir | ah', 50],
                                   [5, 'pc | devs | content | steam | keep', 45],
                                   [6, 'space | ninja | robot | weapon | zombie', 41],
                                   [7, 'awesome | cool | awsome | great | amazing', 39],
                                   [8, 'coop | character | get | multiplayer | beautiful', 35],
                                   [9, 'hour | seems | played | good | yesyesyesyes', 32],
                                   [10, 'player | get | time | feel | make', 31],
                                   [11, 'graphic | story | nice | beautiful | le', 30],
                                   [12, 'free | anthem | better | excellent | best', 30],
                                   [13, 'shooter | person | third | thirdperson | space', 28],
                                   [14, 'fun | haha | ending | leaving | lag', 28],
                                   [15, 'fp | best | game | year | community', 27],
                                   [16, 'game | best | free | freeplay | regret', 27],
                                   [17, 'better | best | approves | waifu | everrrrr', 25],
                                   [18, 'review | new | people | get | bug', 24],
                                   [19, 'love | muffin | clem | pink | sai', 22],
                                   [20, 'currency | platinum | ingame | cosmetic | free', 22],
                                   [21, 'pretty | good | decent | okay | true', 20],
                                   [22, 'digital | extreme | year | playing | time', 20],
                                   [23, 'played | best | game | ive | gamer', 19],
                                   [24, 'farming | simulator | slidey | farm | low', 18],
                                   [25, 'gud | gam | wery | iz | gudye', 15],
                                   [26, 'yes | iz | great |  | ', 15],
                                   [27, 'addictive | addicted | heroin | ruuuunnnn | completley', 14],
                                   [28, 'little | played | seems | kinda | bit', 12],
                                   [29, 'nice | shotted | aliaens | yes | suit', 12],
                                   [30, 'update | year | updating | playing | time', 12],
                                   [31, 'friend | time | spend | account | fun', 10]], dtype=object),
              'hovertemplate': '<b>Topic %{customdata[0]}</b><br>%{customdata[1]}<br>Size: %{customdata[2]}',
              'legendgroup': '',
              'marker': {'color': '#B0BEC5',
                         'line': {'color': 'DarkSlateGrey', 'width': 2},
                         'size': array([60, 54, 52, 52, 50, 45, 41, 39, 35, 32, 31, 30, 30, 28, 28, 27, 27, 25,
                                        24, 22, 22, 20, 20, 19, 18, 15, 15, 14, 12, 12, 12, 10]),
                         'sizemode': 'area',
                         'sizeref': 0.0375,
                         'symbol': 'circle'},
              'mode': 'markers',
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'type': 'scatter',
              'x': array([ 7.9790683 , 11.996285  , -6.6115294 ,  9.977154  , 15.49364   ,
                          12.4494095 , -6.6162667 , 10.091907  , -5.8057995 , 12.050097  ,
                          -6.368161  , -6.76248   , -0.5812504 , -6.9460073 , 15.209008  ,
                          -1.0333234 , -0.78959817, 10.50959   , 12.233138  , 10.596063  ,
                           7.8899884 , 15.336745  , 12.267607  , -1.0360

In [20]:
# 부정 리뷰 토픽 관계도 시각화
topic_model_negative.visualize_topics()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'customdata': array([[0, 'get | player | time | grind | thing', 90],
                                   [1, 'life | send | help | played | stop', 52],
                                   [2, 'hate | bad | hated | wee | bum', 33],
                                   [3, 'review | people | negative | aoe | leave', 19],
                                   [4, 'space | ninja | key | power | janus', 18],
                                   [5, 'addicting | addictive | kill | recommend | ruining', 16],
                                   [6, 'received | product | free | step | achievement', 16]], dtype=object),
              'hovertemplate': '<b>Topic %{customdata[0]}</b><br>%{customdata[1]}<br>Size: %{customdata[2]}',
              'legendgroup': '',
              'marker': {'color': '#B0BEC5',
                         'line': {'color': 'DarkSlateGrey', 'width': 2},
                         'size': array([90, 52, 33, 19, 18, 16, 16]),
                         'sizemode': 'area',
                         'sizeref': 0.05625,
                         'symbol': 'circle'},
              'mode': 'markers',
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'type': 'scatter',
              'x': array([ 9.115844, 12.463014, 11.885875,  8.795968,  8.532217, 12.62397 ,
                           9.406732], dtype=float32),
              'xaxis': 'x',
              'y': array([-11.61115  ,  -4.6148005,  -4.713931 ,  -9.662599 , -11.213022 ,
                           -5.2130165, -10.946643 ], dtype=float32),
              'yaxis': 'y'}],
    'layout': {'annotations': [{'showarrow': False,
                                'text': 'D1',
                                'x': 7.25238447189331,
                                'y': -8.637701320648192,
                                'yshift': 10},
                               {'showarrow': False,
                                'text': 'D2',
                                'x': 10.884975004196168,
                                'xshift': 10,
                                'y': -3.92258038520813}],
               'height': 650,
               'hoverlabel': {'bgcolor': 'white', 'font': {'family': 'Rockwell', 'size': 16}},
               'legend': {'itemsizing': 'constant', 'tracegroupgap': 0},
               'margin': {'t': 60},
               'shapes': [{'line': {'color': '#CFD8DC', 'width': 2},
                           'type': 'line',
                           'x0': 10.884975004196168,
                           'x1': 10.884975004196168,
                           'y0': -13.352822256088256,
                           'y1': -3.92258038520813},
                          {'line': {'color': '#9E9E9E', 'width': 2},
                           'type': 'line',
                           'x0': 7.25238447189331,
                           'x1': 14.517565536499024,
                           'y0': -8.637701320648192,
                           'y1': -8.637701320648192}],
               'sliders': [{'active': 0,
                            'pad': {'t': 50},
                            'steps': [{'args': [{'marker.color': [['red',
                                                                  '#B0BEC5',
                                                                  '#B0BEC5',
                                                                  '#B0BEC5',
                                                                  '#B0BEC5',
                                                                  '#B0BEC5',
                                                                  '#B0BEC5']]}],
                                       'label': 'Topic 0',
                                       'method': 'update'},
                                      {'args': [{'marker.color': [['#B0BEC5',
                                                                  'red', '#B0BEC5',
                                                                  '#B0

In [21]:
# 긍정 리뷰
topic_model_positive.visualize_barchart(top_n_topics=10)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'marker': {'color': '#D55E00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.021873688933877956, 0.024457257076038328,
                    0.08430609281335033, 0.16246694255802344, 0.16323244093185652],
              'xaxis': 'x',
              'y': [best  , played  , free  , product  , received  ],
              'yaxis': 'y'},
             {'marker': {'color': '#0072B2'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.026557271306499125, 0.036388534539078116,
                    0.03745232663626611, 0.08017773360100601, 0.5100401981689908],
              'xaxis': 'x2',
              'y': [good  , fun  , grindy  , grinding  , grind  ],
              'yaxis': 'y2'},
             {'marker': {'color': '#CC79A7'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.015090145770033092, 0.015906892305297984,
                    0.01621569077124849, 0.01721793683229958, 0.02318770882764612],
              'xaxis': 'x3',
              'y': [great  , get  , warframes  , mission  , weapon  ],
              'yaxis': 'y3'},
             {'marker': {'color': '#E69F00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [1e-05, 0.6126781844055612, 0.971185780867055,
                    1.3857075636329332, 2.494547836081482],
              'xaxis': 'x4',
              'y': [  , win  , lol  , antipay  , uwu  ],
              'yaxis': 'y4'},
             {'marker': {'color': '#56B4E9'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.09560463077513963, 0.10223556705251975, 0.1135825871830273,
                    0.1135825871830273, 0.9962024295660868],
              'xaxis': 'x5',
              'y': [ah  , sir  , gamee  , owo  , good  ],
              'yaxis': 'y5'},
             {'marker': {'color': '#009E73'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.026777260836868454, 0.02931697689160406, 0.0310810852210741,
                    0.038556220479021054, 0.038907011142594844],
              'xaxis': 'x6',
              'y': [keep  , steam  , content  , devs  , pc  ],
              'yaxis': 'y6'},
             {'marker': {'color': '#F0E442'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.02564420726435242, 0.036341023397193827,
                    0.03640116696266525, 0.1342362622890981, 0.1613947361904392],
              'xaxis': 'x7',
              'y': [zombie  , weapon  , robot  , ninja  , space  ],
              'yaxis': 'y7'},
             {'marker': {'color': '#D55E00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.20488277732503749, 0.2692555421711787, 0.2787716563822253,
                    0.28492671777964856, 0.5228246132552251],
              'xaxis': 'x8',
              'y': [amazing  , great  , awsome  , cool  , awesome  ],
              'yaxis': 'y8'},
             {'marker': {'color': '#0072B2'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.020369827674071885, 0.021949847971273924,
                    0.022045283033232797, 0.024245064774666394,
                    0.09401446451573456],
              'xaxis': 'x9',
              'y': [beautiful  , multiplayer  , get  , character  , coop  ],
              'yaxis': 'y9'},
             {'marker': {'color': '#CC79A7'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.06267708130857995, 0.06586381222177991, 0.08710593443538776,
                    0.09379136531484197, 0.19900211183058455],
              'xaxis': 'x10',
              'y': [yesyesyesyes  , good  , played  , seems  , hour  ],
              'yaxis': 'y10'}],
    'layout': {'annotations': [{'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Topic 0',
                

In [22]:
# 부정 리뷰
topic_model_negative.visualize_barchart(top_n_topics=10)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'marker': {'color': '#D55E00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.028420450743145257, 0.03127053166356781,
                    0.032297985192263456, 0.03714874402441234, 0.03813723325954622],
              'xaxis': 'x',
              'y': [thing  , grind  , time  , player  , get  ],
              'yaxis': 'y'},
             {'marker': {'color': '#0072B2'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.06534973637992138, 0.0686313049708085, 0.08340710754359353,
                    0.0838593308070725, 0.09699496171577336],
              'xaxis': 'x2',
              'y': [stop  , played  , help  , send  , life  ],
              'yaxis': 'y2'},
             {'marker': {'color': '#CC79A7'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.14521898206930628, 0.14521898206930628, 0.20338763728256784,
                    0.24217046182556712, 0.6497646518298255],
              'xaxis': 'x3',
              'y': [bum  , wee  , hated  , bad  , hate  ],
              'yaxis': 'y3'},
             {'marker': {'color': '#E69F00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.057381329123148445, 0.0630549284223637, 0.08394429466807257,
                    0.08954491213182503, 0.17078731109414555],
              'xaxis': 'x4',
              'y': [leave  , aoe  , negative  , people  , review  ],
              'yaxis': 'y4'},
             {'marker': {'color': '#56B4E9'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.04241797819416948, 0.043181367554175884,
                    0.05084576336019033, 0.07685824886703865, 0.13058971223882798],
              'xaxis': 'x5',
              'y': [janus  , power  , key  , ninja  , space  ],
              'yaxis': 'y5'},
             {'marker': {'color': '#009E73'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.14986457483978682, 0.15928998242605835, 0.167355203786885,
                    0.2553700138905061, 0.44689752430838575],
              'xaxis': 'x6',
              'y': [ruining  , recommend  , kill  , addictive  , addicting  ],
              'yaxis': 'y6'},
             {'marker': {'color': '#F0E442'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.08630253542164801, 0.08630253542164801, 0.22283426191273878,
                    0.23982218485207252, 0.23982218485207252],
              'xaxis': 'x7',
              'y': [achievement  , step  , free  , product  , received  ],
              'yaxis': 'y7'}],
    'layout': {'annotations': [{'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Topic 0',
                                'x': 0.0875,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 1.0,
                                'yanchor': 'bottom',
                                'yref': 'paper'},
                               {'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Topic 1',
                                'x': 0.36250000000000004,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 1.0,
                                'yanchor': 'bottom',
                                'yref': 'paper'},
                               {'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Topic 2',
                                'x': 0.6375000000000001,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 1.0,
                                'yanchor': 'bottom',
  

In [23]:
topic_model_positive.visualize_heatmap()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'coloraxis': 'coloraxis',
              'hovertemplate': 'x: %{x}<br>y: %{y}<br>Similarity Score: %{z}<extra></extra>',
              'name': '0',
              'type': 'heatmap',
              'x': [0_received_product_free, 1_grind_grinding_grindy,
                    2_weapon_mission_warframes, 3_uwu_antipay_lol,
                    4_good_owo_gamee, 5_pc_devs_content, 6_space_ninja_robot,
                    7_awesome_cool_awsome, 8_coop_character_get,
                    9_hour_seems_played, 10_player_get_time, 11_graphic_story_nice,
                    12_free_anthem_better, 13_shooter_person_third,
                    14_fun_haha_ending, 15_fp_best_game, 16_game_best_free,
                    17_better_best_approves, 18_review_new_people,
                    19_love_muffin_clem, 20_currency_platinum_ingame,
                    21_pretty_good_decent, 22_digital_extreme_year,
                    23_played_best_game, 24_farming_simulator_slidey,
                    25_gud_gam_wery, 26_yes_iz_great, 27_addictive_addicted_heroin,
                    28_little_played_seems, 29_nice_shotted_aliaens,
                    30_update_year_updating, 31_friend_time_spend],
              'xaxis': 'x',
              'y': [0_received_product_free, 1_grind_grinding_grindy,
                    2_weapon_mission_warframes, 3_uwu_antipay_lol,
                    4_good_owo_gamee, 5_pc_devs_content, 6_space_ninja_robot,
                    7_awesome_cool_awsome, 8_coop_character_get,
                    9_hour_seems_played, 10_player_get_time, 11_graphic_story_nice,
                    12_free_anthem_better, 13_shooter_person_third,
                    14_fun_haha_ending, 15_fp_best_game, 16_game_best_free,
                    17_better_best_approves, 18_review_new_people,
                    19_love_muffin_clem, 20_currency_platinum_ingame,
                    21_pretty_good_decent, 22_digital_extreme_year,
                    23_played_best_game, 24_farming_simulator_slidey,
                    25_gud_gam_wery, 26_yes_iz_great, 27_addictive_addicted_heroin,
                    28_little_played_seems, 29_nice_shotted_aliaens,
                    30_update_year_updating, 31_friend_time_spend],
              'yaxis': 'y',
              'z': array([[1.        , 0.41660392, 0.47939974, ..., 0.24195124, 0.48546907,
                           0.4504419 ],
                          [0.41660392, 0.9999999 , 0.64322865, ..., 0.26758182, 0.51943135,
                           0.63858575],
                          [0.47939974, 0.64322865, 0.99999976, ..., 0.20755081, 0.5942853 ,
                           0.7239424 ],
                          ...,
                          [0.24195124, 0.26758182, 0.20755081, ..., 1.        , 0.31053668,
                           0.21208522],
                          [0.48546907, 0.51943135, 0.5942853 , ..., 0.31053668, 0.99999994,
                           0.63632375],
                          [0.4504419 , 0.63858575, 0.7239424 , ..., 0.21208522, 0.63632375,
                           1.0000001 ]], dtype=float32)}],
    'layout': {'coloraxis': {'colorbar': {'title': {'text': 'Similarity Score'}},
                             'colorscale': [[0.0, 'rgb(247,252,240)'], [0.125,
                                            'rgb(224,243,219)'], [0.25,
                                            'rgb(204,235,197)'], [0.375,
                                            'rgb(168,221,181)'], [0.5,
                                            'rgb(123,204,196)'], [0.625,
                                            'rgb(78,179,211)'], [0.75,
                                            'rgb(43,140,190)'], [0.875,
                                            'rgb(8,104,172)'], [1.0,
                                            'rgb(8,64,129)']]},
               'height': 800,
               'hoverlabel': {'bgcolor': 'white', 'font': {'family': 'Rockwell', 'size': 16}},
               'legend': {'title': {'tex

In [24]:
topic_model_negative.visualize_heatmap()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'coloraxis': 'coloraxis',
              'hovertemplate': 'x: %{x}<br>y: %{y}<br>Similarity Score: %{z}<extra></extra>',
              'name': '0',
              'type': 'heatmap',
              'x': [0_get_player_time, 1_life_send_help, 2_hate_bad_hated,
                    3_review_people_negative, 4_space_ninja_key,
                    5_addicting_addictive_kill, 6_received_product_free],
              'xaxis': 'x',
              'y': [0_get_player_time, 1_life_send_help, 2_hate_bad_hated,
                    3_review_people_negative, 4_space_ninja_key,
                    5_addicting_addictive_kill, 6_received_product_free],
              'yaxis': 'y',
              'z': array([[0.9999999 , 0.67870337, 0.3840353 , 0.63272834, 0.5959486 ,
                           0.5313339 , 0.53386486],
                          [0.67870337, 0.99999964, 0.55259645, 0.59402907, 0.51838493,
                           0.62319183, 0.50269914],
                          [0.3840353 , 0.55259645, 0.9999998 , 0.45578402, 0.3907876 ,
                           0.46670434, 0.381526  ],
                          [0.63272834, 0.59402907, 0.45578402, 0.9999998 , 0.45239988,
                           0.45349163, 0.5016273 ],
                          [0.5959486 , 0.51838493, 0.3907876 , 0.45239988, 1.        ,
                           0.40143886, 0.32522967],
                          [0.5313339 , 0.62319183, 0.46670434, 0.45349163, 0.40143886,
                           1.        , 0.3860484 ],
                          [0.53386486, 0.50269914, 0.381526  , 0.5016273 , 0.32522967,
                           0.3860484 , 1.        ]], dtype=float32)}],
    'layout': {'coloraxis': {'colorbar': {'title': {'text': 'Similarity Score'}},
                             'colorscale': [[0.0, 'rgb(247,252,240)'], [0.125,
                                            'rgb(224,243,219)'], [0.25,
                                            'rgb(204,235,197)'], [0.375,
                                            'rgb(168,221,181)'], [0.5,
                                            'rgb(123,204,196)'], [0.625,
                                            'rgb(78,179,211)'], [0.75,
                                            'rgb(43,140,190)'], [0.875,
                                            'rgb(8,104,172)'], [1.0,
                                            'rgb(8,64,129)']]},
               'height': 800,
               'hoverlabel': {'bgcolor': 'white', 'font': {'family': 'Rockwell', 'size': 16}},
               'legend': {'title': {'text': 'Trend'}},
               'margin': {'t': 60},
               'showlegend': True,
               'template': '...',
               'title': {'font': {'color': 'Black', 'size': 22},
                         'text': '<b>Similarity Matrix</b>',
                         'x': 0.55,
                         'xanchor': 'center',
                         'y': 0.95,
                         'yanchor': 'top'},
               'width': 800,
               'xaxis': {'anchor': 'y', 'constrain': 'domain', 'domain': [0.0, 1.0], 'scaleanchor': 'y'},
               'yaxis': {'anchor': 'x', 'autorange': 'reversed', 'constrain': 'domain', 'domain': [0.0, 1.0]}}
})